*Script*:
implémentation des techniques de la machine learning pour la prédiction du taux de Churn pour je jeux de données Telecom Costumer

*Description* :
Ce script est dédie à la comparaison entres des algorithmes de classification supervisée et les algorithmes à base d'ensemble pour la prédiction de taux de Churn.
il structuré selon les étapes ci-dessous :
1 - Importation des bibliothèques.
2 - Chargement du jeu de données.
3 - Suppression des variables inutiles.
4 - Traitement des valeurs manquantes et les valeurs erronées.
6 - Transformation des types des variables Objet en des types numériques.
7 - Calcul des coefficients de corrélation entre les différentes variables du jeu de données et la suppréssion d'elles ayant un coéfficient supérieur ou égal à 0,95
8 - Séparation entres les variables prédictives et la variables à prédire.
9 - Normalisation des variables prédictives.
10- Appliquer la technique de la validation croisée pour l'implémentation des algorithmes sélectionnés pour l'étude comparative.
11- Calcul des métriques Accuracy et temps d'execution pour chaque algorithme
12- Calcul de la métrique AUC
13- Affichage de différent graphiques 
14- Tracage de la courbe ROC pour tous les algorithmes de l'étude comparative.

*Version*:
GODIAL KHALID : Juillet 2019 Script Original


In [ ]:
#importation des bibliothèques
import pandas as pd 
import numpy as np
import time
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sklearn.preprocessing import LabelEncoder, OneHotEncoder 
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from catboost import CatBoostClassifier
from sklearn.calibration import CalibratedClassifierCV
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
from scipy import interp


In [ ]:
#Chargement du jeu de données
try:
    ds1=pd.read_csv("../input/Telecom_customer churn.csv")
    print("dataset is loaded")
except:
    print("Dataset could not be loaded. Is the dataset missing?")

In [ ]:
#Traitement des valeurs manquantes 
ds1=ds1.fillna(method='ffill')

In [ ]:
#Suppression des variables inutiles
ds1.pop('Customer_ID')

In [ ]:
#Transformation des types des variables Objet en des types numériques.
labelencoder= LabelEncoder()
colum_Object=ds1.select_dtypes(include=['object'])
for c in colum_Object:
    ds1[c] = labelencoder.fit_transform(ds1[c])

In [ ]:
ds1=ds1.drop(columns=['churn']).assign(churn=ds1['churn'])

In [ ]:
print(ds1['churn'].value_counts())

In [ ]:
# Calcul des coefficients de corrélation entre les différentes variables du jeu de données 
# et la suppréssion d'elles ayant un coéfficient supérieur ou égal à 0,95

#creation de la matrice de corrélation
corr_matrix = ds1.corr().abs()

# sélection du triangle de la matrice de corrélation
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# cherche les indices des variables ayant un coéfficient de corrélation >= 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
#suppression d'une variable ayant un coéfficient de corrélation >=0.95
for c in to_drop:
    ds1.pop(c)

In [ ]:
d=pd.DataFrame({'exclut':to_drop})
d

In [ ]:
ds1.shape

In [ ]:
#Séparation entres les variables prédictives et la variables à prédire.
X=ds1.iloc[:,:-1].values
y=ds1.iloc[:,ds1.columns.size-1].values

In [ ]:
ds1.describe()

In [ ]:
# Normalisation des variables prédictives.
sc_X = StandardScaler()
X = sc_X.fit_transform(X)

In [ ]:
# tableau contient l'abréviation et l'instance de chaque classifieur
classifieurs = []
classifieurs.append(('LR', LogisticRegression(multi_class='ovr', solver='lbfgs', max_iter=1000)))
classifieurs.append(('CART', DecisionTreeClassifier()))
classifieurs.append(('LSVM',  CalibratedClassifierCV(base_estimator=LinearSVC(penalty='l2', dual=False), cv=10)))
classifieurs.append(('RF', RandomForestClassifier(n_estimators=300)))
classifieurs.append(('ADAB', AdaBoostClassifier(n_estimators=300, )))
classifieurs.append(('XGB', XGBClassifier( n_estimators=300, n_jobs=-1)))
classifieurs.append(('LGBM', LGBMClassifier()))
classifieurs.append(('CATB', CatBoostClassifier()))

#Tableau contient les Accuracy de chaque classifieur
results = []
#Tableau contient le nom de chaque classifieur
names = []
#Tableau contient le temps d'execution pour chaque classifieur
times =[]
#Tableau contient l'Accuracy moyenne pour chaque classifieur
results_mean=[]
#Tableau contient l'écart-type moyenne pour chaque classifieur
results_std=[]

# Métrique d'évaluation a calculée
scoring = 'accuracy'

for name, model in classifieurs:
    kfold = model_selection.KFold(n_splits=10, random_state=0)
    start = time.time()
    cv_results = model_selection.cross_val_score(model, X, y, cv=10, scoring='accuracy')
    end = time.time()
    t=(end-start)/10
    times.append(t)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (+/- %f) : %f s" % (name, cv_results.mean(), cv_results.std(),t)
    results_mean.append(cv_results.mean())
    results_std.append(cv_results.std())

In [ ]:
# implémentation dex réseaux des neurones
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
cvscores = []
start = time.time()
for train, test in kfold.split(X, y):
    classifieur = Sequential()
    classifieur.add(Dense((ds1.columns.size-1)*2, input_dim=ds1.columns.size-1, activation='relu'))
    classifieur.add(Dense((ds1.columns.size-1), activation='relu'))
    classifieur.add(Dense(1, activation='sigmoid'))
    classifieur.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    classifieur.fit(X[train], y[train], epochs=10, batch_size=10, verbose=0)
    scores = classifieur.evaluate(X[test], y[test],   verbose=0)
    print("%s: %.2f%%" % (classifieur.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1])
end = time.time()
t=(end-start)/10
times.append(t)
results.append(cvscores)
results_mean.append(np.mean(cvscores))
results_std.append(np.std(cvscores))
names.append('DL')

In [ ]:
results=list(map(list, zip(*results)))
r = pd.DataFrame(results,columns=names)
r

In [ ]:
results=list(map(list, zip(*results)))

In [ ]:

k_range = range(1, 11)
plt.style.use('seaborn-whitegrid')
for i in range(0,8):
    plt.plot(k_range, results[i],marker='o', markerfacecolor='blue', markersize=12, color='skyblue', linewidth=4, linestyle='dashed')
    plt.xlabel('K-fold')
    plt.ylabel('Cross-Validated Accuracy')
    plt.title(names[i])
    plt.show()

In [ ]:
plt.figure(1, figsize = (8,5) )
plt.style.use('seaborn-whitegrid')
plt.bar(names,times, color='red',width=0.1)
plt.xlabel('Classifieurs')
plt.ylabel('Temps execution en s')
plt.show()
plt.figure(1, figsize = (8,5) )

plt.errorbar(names, results_mean, yerr=results_std, fmt='ro', ecolor='blue',elinewidth=5, capsize=10)
plt.xlabel('Classifieurs')
plt.ylabel('Accuracy avec std')
plt.show()

In [ ]:
# calcul de AUC
results_Roc_Auc = []
names_Roc_Auc = []
results_Roc_Auc_mean=[]
results_Roc_Auc_std=[]
names_Roc_Auc=[]
scoring = 'roc_auc'
for name, model in classifieurs:
    kfold = model_selection.KFold(n_splits=10, random_state=0)
    cv_results_Roc_Auc = model_selection.cross_val_score(model, X, y, cv=kfold, scoring=scoring)
    results.append(cv_results_Roc_Auc)
    names_Roc_Auc.append(name)
    msg = "%s: %.2f (+/- %f)" % (name, cv_results_Roc_Auc.mean(),cv_results_Roc_Auc.std())
    results_Roc_Auc_mean.append(cv_results_Roc_Auc.mean())
    results_Roc_Auc_std.append(cv_results_Roc_Auc.std())

In [ ]:
# calcul de AUC pour Deep learning
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
cvscores_Roc_Auc = []
for train, test in kfold.split(X, y):
    classifieur = Sequential()
    classifieur.add(Dense((ds1.columns.size-1)*2, input_dim=ds1.columns.size-1, activation='relu'))
    classifieur.add(Dense(ds1.columns.size-1, activation='relu'))
    classifieur.add(Dense(1, activation='sigmoid'))
    classifieur.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    classifieur.fit(X[train], y[train], epochs=10, batch_size=10, verbose=0)
    scores = classifieur.evaluate(X[test], y[test],   verbose=0)
    y_pred = classifieur.predict_proba(X[test])
    cvscores_Roc_Auc.append(roc_auc_score(y[test], y_pred))
results_Roc_Auc_mean.append(np.mean(cvscores_Roc_Auc))
results_Roc_Auc_std.append(np.std(cvscores_Roc_Auc))
names_Roc_Auc.append('DL')
msg = "DL: %.2f (+/- %f)" % (np.mean(cvscores_Roc_Auc),np.std(cvscores_Roc_Auc))

In [ ]:
plt.figure(1, figsize = (8,5) )
plt.style.use('seaborn-whitegrid')
plt.errorbar(names, results_Roc_Auc_mean, yerr=results_Roc_Auc_std, fmt='ro', ecolor='blue',elinewidth=5, capsize=10)
plt.xlabel('Classifieurs')
plt.ylabel('AUC avec std')
plt.show()

In [ ]:
AC_AUC = pd.DataFrame({'Accuracy':results_mean,'AUC':results_Roc_Auc_mean}, index=names)
AC_AUC.plot(kind='bar')

In [ ]:
df_all = pd.DataFrame({'Temps':times,'Accuracy':results_mean,'AUC':results_Roc_Auc_mean}, index=names)
df_all

In [ ]:
# traçage des courbes ROC pour tous les classifieurs

classifieurs = []
classifieurs.append(('LR', LogisticRegression(multi_class='ovr', solver='lbfgs', max_iter=1000)))
classifieurs.append(('CART', DecisionTreeClassifier()))
classifieurs.append(('LSVM',  CalibratedClassifierCV(base_estimator=LinearSVC(penalty='l2', dual=False), cv=10)))
classifieurs.append(('RF', RandomForestClassifier(n_estimators=300)))
classifieurs.append(('ADAB', AdaBoostClassifier(n_estimators=300, )))
classifieurs.append(('XGB', XGBClassifier( n_estimators=300, n_jobs=-1)))
classifieurs.append(('LGBM', LGBMClassifier()))
classifieurs.append(('CATB', CatBoostClassifier()))

# plot arrows
fig1 = plt.figure(figsize=[12,12])
ax1 = fig1.add_subplot(111,aspect = 'equal')

tprs = []
aucs = []
mean_fpr = np.linspace(0,1,100)
c=['blue','green','red','cyan','magenta','yellow','violet','grey']
j=0
for name, model in classifieurs:
    kfold = model_selection.KFold(n_splits=10, random_state=0)
    cv=kfold
    i = 1
    for train,test in cv.split(X,y):
        prediction = model.fit(X[train],y[train]).predict_proba(X[test])
        fpr, tpr, t = roc_curve(y[test], prediction[:, 1])
        tprs.append(interp(mean_fpr, fpr, tpr))
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)
        i= i+1
    plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
    mean_tpr = np.mean(tprs, axis=0)
    mean_auc = auc(mean_fpr, mean_tpr)
    plt.plot(mean_fpr, mean_tpr, color=c[j],label=r'%s (AUC = %0.2f )' % (name,mean_auc),lw=2, alpha=1)
    j=j+1
    
for train, test in kfold.split(X, y):
    classifieur = Sequential()
    classifieur.add(Dense((ds1.columns.size-1)*2, input_dim=ds1.columns.size-1, activation='relu'))
    classifieur.add(Dense(ds1.columns.size-1, activation='relu'))
    classifieur.add(Dense(1, activation='sigmoid'))
    classifieur.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    classifieur.fit(X[train], y[train], epochs=10, batch_size=3, verbose=0)
    prediction = classifieur.predict_proba(X[test])
    fpr, tpr, t = roc_curve(y[test], prediction)
    tprs.append(interp(mean_fpr, fpr, tpr))
    roc_auc = auc(fpr, tpr)
    aucs.append(roc_auc)
plt.plot([0,1],[0,1],linestyle = '--',lw = 2,color = 'black')
mean_tpr = np.mean(tprs, axis=0)
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='orange',label=r'DL (AUC = %0.2f )' % (mean_auc),lw=2, alpha=1)

plt.xlabel('Taux Faux Positif')
plt.ylabel('Taux Vrai Positif')
plt.title('ROC')
plt.legend(loc="lower right")
plt.show()